In [1]:
import torch
import datetime
import os
from sklearn.metrics import mean_squared_error, roc_auc_score, f1_score
from fl_simu import lstm_simulation2

D:\FD\venv\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
print('main2_lstm_federated')

main2_lstm_federated


In [3]:
# Load data
nts = torch.load('./dataset2/nts.pt')
y = torch.load('./dataset2/y.pt')
ts1 = torch.load('./dataset2/ts1.pt')
ts2 = torch.load('./dataset2/ts2.pt')
ts3 = torch.load('./dataset2/ts3.pt')
ts4 = torch.load('./dataset2/ts4.pt')

In [4]:
def test(net, x_test, y_test):
  with torch.no_grad():
    y_pred = net(x_test)
    y_pred = y_pred.detach().numpy()
    y_test = y_test.detach().numpy()

    mse = mean_squared_error(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    return mse, auc

In [5]:
# Define LSTM FL simulation wrapper
def run_lstm_simulation(split, nts_train, ts1_train, ts2_train, ts3_train, ts4_train, y_train, nts_test, ts1_test, ts2_test, ts3_test, ts4_test, y_test,dir='default', num_epoch=4, batch_size=128, num_rounds=5, dropout_rate=False):
    return lstm_simulation2(split, nts_train, ts1_train, ts2_train, ts3_train, ts4_train, y_train, nts_test, ts1_test, ts2_test, ts3_test, ts4_test, y_test, dir, num_epoch, batch_size, num_rounds, dropout_rate)

In [6]:
from sklearn.model_selection import StratifiedKFold

batch_size = 128

def do_fl(splitting, nts=nts, y=y, ts1=ts1, ts2=ts2, ts3=ts3, ts4=ts4,num_rounds=10,num_epoch = 20):
    mses = []
    aucs = []

    test_name = 'Dataset 2 LSTM'

    directory_path = 'lstm_model_checkpoints2/' + test_name

    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

    start = datetime.datetime.now()

    skf = StratifiedKFold(n_splits=5)
    for fold_idx, (train_idx, test_idx) in enumerate(skf.split(nts, y)):
        print(f"Fold {fold_idx + 1}:")

        nts_train, nts_test = nts[train_idx], nts[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        ts1_train = ts1[train_idx]
        ts2_train = ts2[train_idx]
        ts3_train = ts3[train_idx]
        ts4_train = ts4[train_idx]
        ts1_test = ts1[test_idx]
        ts2_test = ts2[test_idx]
        ts3_test = ts3[test_idx]
        ts4_test = ts4[test_idx]

        model_fl = run_lstm_simulation(splitting, nts_train, ts1_train, ts2_train, ts3_train, ts4_train, y_train, nts_test, ts1_test, ts2_test, ts3_test, ts4_test, y_test,test_name, num_epoch=num_epoch, num_rounds=num_rounds, batch_size=batch_size)

        mse, auc = test(model_fl, (nts_test, ts1_test, ts2_test, ts3_test, ts4_test), y_test)
        mses.append(mse)
        aucs.append(auc)

    end = datetime.datetime.now()
    time = end - start

    print('splitting:', splitting)
    print('num_epochs', num_epoch)
    print('num_rounds', num_rounds)
    print('mse:', mse)
    print('auc: ', auc)
    print('Training time: ', time)

In [ ]:
splitting = [0.5, 0.5]
do_fl(splitting, num_rounds = 1)

Fold 1:


INFO flwr 2023-08-30 23:17:08,793 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=1, round_timeout=None)


In [ ]:
# splitting = [0.6, 0.4]
# do_fl(splitting,num_rounds = 10)

In [ ]:
# splitting = [0.9, 0.1]
# do_fl(splitting,num_rounds = 10)

In [ ]:
# splitting = [0.2, 0.2, 0.2, 0.2, 0.2]
# do_fl(splitting, num_rounds = 20)

In [ ]:
# splitting = [0.6, 0.1, 0.1, 0.1, 0.1]
# do_fl(splitting, num_rounds = 20)

In [ ]:
# splitting = [0.8, 0.05, 0.05, 0.05, 0.05]
# do_fl(splitting, num_rounds = 20)

In [ ]:
# splitting = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
# do_fl(splitting,num_rounds = 20)

In [ ]:
# split = [0.6, 0.05, 0.05, 0.05, 0.05, 0.04, 0.04, 0.04, 0.04, 0.04]
# do_fl(splitting,num_rounds = 20)

In [ ]:
# splitting = [0.8, 0.03, 0.03, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02]
# do_fl(splitting,num_rounds = 20)